In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import os
os.environ["KERAS_BACKEND"] = "tensorflow" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/math-qsa-dataset/train.csv
/kaggle/input/math-qsa-dataset/test.csv
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/config.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/tokenizer.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/metadata.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/model.weights.h5
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/assets/tokenizer/vocabulary.spm
/kaggle/input/keras-lib-dataset/keras-3.3.3-py3-none-any.whl
/kaggle/input/keras-lib-dataset/keras_nlp-0.12.1-py3-none-any.whl
/kaggle/input/ai-mathematical-olympiad-prize/sample_submission.csv
/kaggle/input/ai-mathematical-olympiad-prize/AIMO Prize - Note on Language and Notation.pdf
/kaggle/input/ai-mathematical-olympiad-prize/train.csv
/kaggle/input/ai-mathematical-olympiad-prize/test.csv
/kaggle/input/ai-mathematical-olympiad-prize/aimo/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/ai-mathematical-olympiad-prize/aimo/__init__.py


2024-06-06 06:57:07.374952: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 06:57:07.375043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 06:57:07.494614: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
!pip install -q /kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl --no-deps
!pip install pylatexenc

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=92b64ae10541984aa0f3764985c5753b6a499149fdb669a018fdc9bf315aac36
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [10]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/ai-mathematical-olympiad-prize"
    preset = "gemma_1.1_instruct_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 1 # number of epochs to train
keras.utils.set_random_seed(CFG.seed)

In [13]:
df1 = pd.read_csv("/kaggle/input/math-qsa-dataset/train.csv")
df2 = pd.read_csv("/kaggle/input/math-qsa-dataset/test.csv")
df = pd.concat([df1, df2], axis=0)

In [14]:
def is_integer(text):
    try:
        if int(text) >= 0:
            return True
        else:
            return False
    except ValueError:
        return False
    
df["is_integer"] = df.answer.map(is_integer)
df = df[df.is_integer].reset_index(drop=True)

# Preprocessing Pipeline

In [15]:
from pylatexenc.latex2text import LatexNodes2Text

def latexconvertor(problems):
    v = []
    for i in range(len(problems)):
        latex = problems[i]
        text = LatexNodes2Text().latex_to_text(latex)
        v.append(text)
    return v


data = latexconvertor(df["problem"])
        

In [2]:
import nltk

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize




stop_words = set(stopwords.words('english'))


word_tokens = word_tokenize(data[4])


filtered_sentence = [word for word in word_tokens if word.lower() not in stop_words]


filtered_text = ' '.join(filtered_sentence)


print("Original Text: ", data[4])
print("Filtered Text: ", filtered_text)


Original Text:  Regular pentagon ABCDE and regular hexagon AEFGHI are drawn on opposite sides of line segment AE such that they are coplanar. What is the degree measure of exterior angle DEF? [asy]
draw((0,2.5)–(0,7.5)–(4,10)–(8,7.5)–(8,2.5)–(4,0)–cycle,linewidth(1));
draw((8,2.5)–(11.5,-1)–(9,-5)–(5,-4.5)–(4,0),linewidth(1));
dot((0,2.5)); dot((0,7.5)); dot ((4,10)); dot((8,7.5)); dot((8,2.5)); dot((4,0));

label("I",(0,2.5),W); label("H",(0,7.5),W); label("G",(4,10),N);

label("F",(8,7.5),E); label("E",(8,2.5),NW); label("A",(4,0),SW);
dot((11.5,-1)); dot((9,-5)); dot((5,-4.5));
label("D",(11.5,-1),E); label("C",(9,-5),SE); label("B",(5,-4.5),SW);
[/asy]
Filtered Text:  Regular pentagon ABCDE regular hexagon AEFGHI drawn opposite sides line segment AE coplanar . degree measure exterior angle DEF ? [ asy ] draw ( ( 0,2.5 ) – ( 0,7.5 ) – ( 4,10 ) – ( 8,7.5 ) – ( 8,2.5 ) – ( 4,0 ) –cycle , linewidth ( 1 ) ) ; draw ( ( 8,2.5 ) – ( 11.5 , -1 ) – ( 9 , -5 ) – ( 5 , -4.5 ) – ( 4,0 ) , linew

In [ ]:
def 

In [16]:
data

['The United States Postal Service charges an extra $0.11 in postage if the length of an envelope, in inches, divided by its height, in inches, is less than 1.3 or greater than 2.5. For how many of these four envelopes must the extra $0.11 in postage be paid? t]ccc\nEnvelope     Length in inches     Height in inches\nA    6    4\n\nB    9    3\n\nC    6    6\n\nD    11    4\n',
 'How many integers between 1000 and 2000 have all three of the numbers 15, 20 and 25 as factors?',
 'Given that n is an integer and 0 < 4n <30, what is the sum of all possible integer values of n?',
 'How many integers between 100 and 150 have three different digits in increasing order? One such integer is 129.',
 'Regular pentagon ABCDE and regular hexagon AEFGHI are drawn on opposite sides of line segment AE such that they are coplanar. What is the degree measure of exterior angle DEF? [asy]\ndraw((0,2.5)–(0,7.5)–(4,10)–(8,7.5)–(8,2.5)–(4,0)–cycle,linewidth(1));\ndraw((8,2.5)–(11.5,-1)–(9,-5)–(5,-4.5)–(4,0),l

# Prompt Engineering

In [9]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

In [10]:
df["prompt"] = df.progress_apply(lambda row: template.format(problem=row.problem,
                                                             solution=f"{row.solution}\n\nAnswer:\n{row.answer}"),
                                                             axis=1)
data = df.prompt.tolist()

  0%|          | 0/7356 [00:00<?, ?it/s]

In [11]:
def colorize_text(text):
    for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
                           ["blue", "yellow", "red", "cyan", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [12]:
# Take a random sample
sample = data[12]

# Give colors to Instruction, Response and Category
sample = colorize_text(sample)

# Show sample in markdown
display(Markdown(sample))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.



**<font color='red'>Problem:</font>**
What is the largest positive multiple of $12$ that is less than $350?$



**<font color='cyan'>Solution:</font>**
Dividing $350$ by $12$ gives a quotient $29$ with a remainder of $2$. In other words, \[350=12\cdot29+2.\]Thus, $29\cdot12=\boxed{348}$ is the largest multiple of $12$ which is less than $350.$



**<font color='green'>Answer:</font>**
348